<h3> Capstone Project <h3>
   

<h4> Introduction/Business problem <h4>

The problem: Which London underground stations could be classified as residential, leisure or business?

Knowing the answer to this problem could help a start up business to choose which area to open in. Or perhaps a town planner to understand the flow of the population and so to extend the rail system in a practical manner.

The data used for this project will be a list of London undergrounds and their coordiantes from https://wiki.openstreetmap.org/wiki/List_of_London_Underground_stations. By exploring the neighrbourhood of each station using the Foursquare API we can find what tyes of venues are in each stations vicinity.For example see that they are mostly 'shop and service' venues then we could categorise the station as leisure and similarly for residential or business. The Foursquare API has it's own subcategories already that we will be basing this categorisation on.

    Arts & Entertainment
    College & University
    Event
    Food
    Nightlife Spot
    Outdoors & Recreation
    Professional & Other Places
    Residence
    Shop & Service
    Travel & Transport
    
To precisely decide which category each station will be in, we will count the venues and note their categories or a certain radius around each station. Then use k-means clustering on the data to categorise each station. 
   